# Wannier WorkGraph

## Minimal workflow

In [6]:
from workgraph_collections.qe.wannier_minimal_base import wannier_minimal_base_workgraph

wannier_wg = wannier_minimal_base_workgraph()
wannier_wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

## Prepare inputs and submit


In [5]:

from workgraph_collections.qe.wannier_minimal_base import wannier_minimal_base_workgraph
from aiida_wannier90.workflows.minimal import get_explicit_kpoints
from aiida import load_profile, orm
from ase.build import bulk
from copy import deepcopy
from aiida.plugins import DataFactory
from aiida_wannier90.orbitals import generate_projections

load_profile()

pw_code = orm.load_code("qe-7.2-pw@localhost")
pw2wannier90_code = orm.load_code('qe-7.2-pw2wannier90@localhost')
wannier90_code = orm.load_code('wannier90@localhost')

StructureData = DataFactory('core.structure')
a = 5.68018817933178
structure = StructureData(cell = [[-a/2., 0, a/2.], [0, a/2., a/2.], [-a/2., a/2., 0]])
structure.append_atom(symbols=['Ga'], position=(0., 0., 0.))
structure.append_atom(symbols=['As'], position=(-a/4., a/4., a/4.))
structure.store()

# Load the pseudopotential family.
pseudo_family = orm.load_group("SSSP/1.3/PBEsol/efficiency")
pseudos = pseudo_family.get_pseudos(structure=structure)

scf_paras = {
    "CONTROL": {
        "calculation": "scf",
    },
    "SYSTEM": {
        "ecutwfc": 30,
        "ecutrho": 240,
        "occupations": "fixed",
    },
    "ELECTRONS": {}
}
nscf_paras = deepcopy(scf_paras)
nscf_paras['CONTROL']['calculation'] = 'nscf'
nscf_paras['SYSTEM'].update({'nosym': True,
                             'noinv': True,
                             })
nscf_paras['ELECTRONS'].update({'startingpot': 'file',
                                'diago_full_acc': True})


scf_kpoints = orm.KpointsData()
scf_kpoints.set_kpoints_mesh([4, 4, 4])
# Use explicit list of kpoints generated by wannier.
# Since the QE auto generated kpoints might be different from wannier90, here we explicitly
# generate a list of kpoint coordinates to avoid discrepancies.
nscf_kpoints = orm.KpointsData()
nscf_kpoints.set_kpoints_mesh([10, 10, 10])
kpoints_nscf_explicit = get_explicit_kpoints(nscf_kpoints)
# If wannier inputs.kpoints is a kmesh, mp_grid will be auto-set by `Wannier90Calculation`,
# otherwise we need to set it manually. If use open_grid, kpoints will be set dynamically
# after open_grid calculation.
mp_grid = nscf_kpoints.get_kpoints_mesh()[0]

# k-points path for the band structure
kpoint_path = orm.Dict(
    {
        "point_coords": {
            "G": [0.0, 0.0, 0.0],
            "K": [0.375, 0.375, 0.75],
            "L": [0.5, 0.5, 0.5],
            "U": [0.625, 0.25, 0.625],
            "W": [0.5, 0.25, 0.75],
            "X": [0.5, 0.0, 0.5],
        },
        "path": [
            ("G", "X"),
            ("X", "U"),
            ("K", "G"),
            ("G", "L"),
            ("L", "W"),
            ("W", "X"),
        ],
    }
)
# sp^3 projections, centered on As
projections = generate_projections(
    {
        "position_cart": (-a / 4.0, a / 4.0, a / 4.0),
        "ang_mtm_l_list": -3,
        "spin": None,
        "spin_axis": None,
    },
    structure=structure,
)
#
metadata = {
    "options": {
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        },
    }
}

# ===============================================================================

wannier_inputs = {
    'seekpath': {"structure": structure},
    'scf': {
        "pw": {
            'code': pw_code,
            'pseudos': pseudos,
            'parameters': orm.Dict(scf_paras),
            'metadata': metadata,
            },
        'kpoints': scf_kpoints,
    },
    'nscf': {
        "pw": {
            'code': pw_code,
            'pseudos': pseudos,
            'parameters': orm.Dict(nscf_paras),
            'metadata': metadata,
        },
        'kpoints': kpoints_nscf_explicit,
    },
    'wannier90_pp': {
        'wannier90': {
            'code': wannier90_code,
            'parameters': orm.Dict({
                "write_hr": False,
                "write_xyz": False,
                "use_ws_distance": True,
                "bands_plot": True,
                "num_iter": 200,
                "guiding_centres": False,
                "num_wann": 4,
                "exclude_bands": [1, 2, 3, 4, 5],
                "mp_grid": mp_grid,
            }),
            'kpoints': kpoints_nscf_explicit,
            "kpoint_path": kpoint_path,
            "projections": projections,
            'metadata': metadata,
            'settings': orm.Dict({'postproc_setup': True}),
        }
    },
    'pw2wannier90': {
        'pw2wannier90': {
            'code': pw2wannier90_code,
            "parameters": orm.Dict({
                "inputpp": {
                    "write_mmn": True,
                    "write_amn": True,
                    "write_unk": True,
                },
            }),
            'metadata': metadata,
        }
    },
    'wannier90': {
        'wannier90': {
            'code': wannier90_code,
            'parameters': orm.Dict({
                "mp_grid": mp_grid,
                "write_hr": False,
                "write_xyz": False,
                "use_ws_distance": True,
                "bands_plot": True,
                "num_iter": 200,
                "guiding_centres": False,
                "num_wann": 4,
                "exclude_bands": [1, 2, 3, 4, 5],
                }),
            'kpoints': kpoints_nscf_explicit,
            "kpoint_path": kpoint_path,
            "projections": projections,
            'metadata': metadata,
            'settings': orm.Dict({'postproc_setup': False}),
        }
    },
    
}
wg = wannier_minimal_base_workgraph(structure=structure, inputs=wannier_inputs)
wg.name = "Wannier-GaAs"
wg.run()


05/28/2024 04:21:56 PM <3373554> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15162|WorkGraphEngine|continue_workgraph]: Continue workgraph.


Continue workgraph.


05/28/2024 04:21:57 PM <3373554> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15162|WorkGraphEngine|continue_workgraph]: nodes ready to run: scf,wannier90_pp
05/28/2024 04:21:57 PM <3373554> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15162|WorkGraphEngine|run_nodes]: Run node: scf, type: WORKCHAIN


------------------------------------------------------------
executor:  <class 'aiida_quantumespresso.workflows.pw.base.PwBaseWorkChain'>
args:  [{'code': <InstalledCode: Remote code 'qe-7.2-pw' on localhost pk: 687, uuid: 1a272f2f-ea8f-48a7-a1ed-cce982f322f2>, 'metadata': {'options': {'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 1}}}, 'parameters': <Dict: uuid: deba6e36-d91e-4126-8616-c565f5ce5fc5 (pk: 15152)>, 'pseudos': {'As': <UpfData: uuid: eaa62681-0915-4398-b70b-61d91bf1f8b3 (pk: 836)>, 'Ga': <UpfData: uuid: 8838818d-d279-433f-ac82-1e6a926e4384 (pk: 818)>}, 'structure': <StructureData: uuid: c8940de2-5546-4965-a0ee-ffdd8de151d6 (pk: 15148)>}, None, None, {}]
kwargs:  {'kpoints': <KpointsData: uuid: 144b2121-4d82-4886-a914-cc06c7077b6c (pk: 15153)>, 'pw': {'code': <InstalledCode: Remote code 'qe-7.2-pw' on localhost pk: 687, uuid: 1a272f2f-ea8f-48a7-a1ed-cce982f322f2>, 'metadata': {'options': {'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 1}}}, 'par

05/28/2024 04:21:57 PM <3373554> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15162|WorkGraphEngine|run_nodes]: Run node: wannier90_pp, type: WORKCHAIN


...

05/28/2024 04:25:47 PM <3373554> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15162|WorkGraphEngine|continue_workgraph]: Continue workgraph.


Continue workgraph.


05/28/2024 04:25:47 PM <3373554> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15162|WorkGraphEngine|continue_workgraph]: nodes ready to run: 
05/28/2024 04:25:47 PM <3373554> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15162|WorkGraphEngine|finalize]: Finalize


node:  nscf FINISHED
node:  pw2wannier90 FINISHED
node:  scf FINISHED
node:  wannier90 FINISHED
node:  wannier90_pp FINISHED
is workgraph finished:  True
group outputs:  []
Finalize workgraph Wannier-GaAs



{'group_outputs': {},
 'new_data': {},
 'execution_count': <Int: uuid: 61079bab-5cbe-4cf2-9e5d-d73cf8e9e10b (pk: 15217) value: 0>}

## Bands workgraph

In [7]:
from workgraph_collections.qe.wannier90 import wannier_workgraph

wannier_wg = wannier_workgraph()
wannier_wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

In [8]:
from workgraph_collections.qe.wannier90 import wannier_workgraph
from aiida import load_profile, orm
from ase.build import bulk
from copy import deepcopy
from aiida.plugins import DataFactory
from aiida_wannier90_workflows.utils.kpoints import (
            create_kpoints_from_distance,
            get_explicit_kpoints,
        )
from aiida_wannier90_workflows.utils.pseudo import (
            get_number_of_projections,
            get_pseudo_and_cutoff,
            get_wannier_number_of_bands,
        )
load_profile()


pw_code = orm.load_code("qe-7.2-pw@localhost")
projwfc_code = orm.load_code('qe-7.2-projwfc@localhost')
pw2wannier90_code = orm.load_code('qe-7.2-pw2wannier90@localhost')
wannier90_code = orm.load_code('wannier90@localhost')

StructureData = DataFactory('core.structure')
a = 5.68018817933178
structure = StructureData(cell = [[-a/2., 0, a/2.], [0, a/2., a/2.], [-a/2., a/2., 0]])
structure.append_atom(symbols=['Ga'], position=(0., 0., 0.))
structure.append_atom(symbols=['As'], position=(-a/4., a/4., a/4.))
structure.store()

# Load the pseudopotential family.
pseudo_family = orm.load_group("SSSP/1.3/PBEsol/efficiency")
pseudos = pseudo_family.get_pseudos(structure=structure)
pseudos, _, _ = get_pseudo_and_cutoff("SSSP/1.3/PBEsol/efficiency", structure)
# INSULATOR
num_bands = get_wannier_number_of_bands(
            structure=structure,
            pseudos=pseudos,
            factor=1.2,
            only_valence=True,
            spin_polarized=False,
            spin_orbit_coupling=False,
        )
num_projs = get_number_of_projections(
    structure=structure,
    pseudos=pseudos,
    spin_orbit_coupling=False,
)
print("num_bands", num_bands)
print("num_projs", num_projs)
num_wann = num_bands # if insulator, num_wann = num_bands else num_wann = num_projs

scf_paras = {
    "CONTROL": {
        "calculation": "scf",
    },
    "SYSTEM": {
        "ecutwfc": 30,
        "ecutrho": 240,
        "occupations": "fixed",
    },
    "ELECTRONS": {}
}
nscf_paras = deepcopy(scf_paras)
nscf_paras['CONTROL']['calculation'] = 'nscf'
nscf_paras['SYSTEM'].update({'nosym': True,
                             'noinv': True,
                             'nbnd': num_bands})
nscf_paras['ELECTRONS'].update({'startingpot': 'file',
                                'diago_full_acc': True})


scf_kpoints = orm.KpointsData()
scf_kpoints.set_kpoints_mesh([2, 2, 2])
# Use explicit list of kpoints generated by wannier.
# Since the QE auto generated kpoints might be different from wannier90, here we explicitly
# generate a list of kpoint coordinates to avoid discrepancies.
kpoints = create_kpoints_from_distance(
            structure,
            distance=0.5,
            force_parity=False,
        )
wannier90_kpoints = get_explicit_kpoints(kpoints)
# If wannier inputs.kpoints is a kmesh, mp_grid will be auto-set by `Wannier90Calculation`,
# otherwise we need to set it manually. If use open_grid, kpoints will be set dynamically
# after open_grid calculation.
mp_grid = kpoints.get_kpoints_mesh()[0]

#
metadata = {
    "options": {
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        },
    }
}

# ===============================================================================

wannier_inputs = {
    'seekpath': {"structure": structure},
    'scf': {
        'pw': {
            'code': pw_code,
            'pseudos': pseudos,
            'parameters': orm.Dict(scf_paras),
            'metadata': metadata,
        },
        'kpoints': scf_kpoints,
    },
    'nscf': {
        'pw': {
            'code': pw_code,
            'pseudos': pseudos,
            'parameters': orm.Dict(nscf_paras),
            'metadata': metadata,
        },
        'kpoints': wannier90_kpoints,
    },
    'projwfc': {
        'projwfc': {
            'code': projwfc_code,
            'metadata': metadata,
        }
    },
    'wannier90_pp': {
        'wannier90': {
            'code': wannier90_code,
            'parameters': orm.Dict({
                "auto_projections": True,
                "bands_plot": True,
                "conv_tol": 2.0e-05,
                "conv_window": 1,
                "dis_conv_tol": 2.0e-05,
                "dis_num_iter": 0,
                "num_cg_steps": 200,
                "num_iter": 400,
                "num_wann": num_wann,
                "num_bands": num_bands,
                "mp_grid": mp_grid,
            }),
            'kpoints': wannier90_kpoints,
            'metadata': metadata,
            'settings': orm.Dict({'postproc_setup': True}),
        },
    },
    'pw2wannier90': {
        'pw2wannier90': {
            'code': pw2wannier90_code,
            "parameters": orm.Dict({
                "inputpp": {
                    "scdm_entanglement": "erfc",
                    "write_mmn": True,
                    "write_amn": True,
                    "write_unk": True,
                    "scdm_proj": True,
                },
            }),
            'metadata': metadata,
        }
    },
    'wannier90': {
        'wannier90': {
            'code': wannier90_code,
            'parameters': orm.Dict({
                "auto_projections": True,
                "bands_plot": True,
                "conv_tol": 2.0e-05,
                "conv_window": 1,
                "dis_conv_tol": 2.0e-05,
                "dis_num_iter": 0,
                "num_cg_steps": 200,
                "num_iter": 400,
                "num_wann": num_wann,
                "num_bands": num_bands,
                "mp_grid": mp_grid,
                }),
            'kpoints': wannier90_kpoints,
            'metadata': metadata,
            'settings': orm.Dict({'postproc_setup': False}),
        },
    },
    
}
wg = wannier_workgraph(structure=structure, inputs=wannier_inputs)
wg.name = "Wannier-GaAs"
wg.run()


num_bands 9
num_projs 13


05/28/2024 04:30:07 PM <3373554> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15228|WorkGraphEngine|continue_workgraph]: Continue workgraph.


Continue workgraph.


05/28/2024 04:30:08 PM <3373554> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15228|WorkGraphEngine|continue_workgraph]: nodes ready to run: seekpath
05/28/2024 04:30:08 PM <3373554> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15228|WorkGraphEngine|run_nodes]: Run node: seekpath, type: CALCFUNCTION


------------------------------------------------------------
executor:  <function seekpath_structure_analysis at 0x7f9c21bbf010>
args:  [<StructureData: uuid: ea4eb09a-86ad-4b50-ac0e-f76f0e572b06 (pk: 15218)>]
kwargs:  {'structure': <StructureData: uuid: ea4eb09a-86ad-4b50-ac0e-f76f0e572b06 (pk: 15218)>}
var_kwargs:  {}
node type: calcfunction/workfunction.


...

05/28/2024 04:31:58 PM <3373554> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15228|WorkGraphEngine|set_node_result]: Node: wannier90 finished.
05/28/2024 04:32:01 PM <3373554> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15228|WorkGraphEngine|continue_workgraph]: Continue workgraph.


Continue workgraph.


05/28/2024 04:32:02 PM <3373554> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15228|WorkGraphEngine|continue_workgraph]: nodes ready to run: 
05/28/2024 04:32:03 PM <3373554> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15228|WorkGraphEngine|finalize]: Finalize


node:  inspect_seekpath FINISHED
node:  nscf FINISHED
node:  projwfc FINISHED
node:  pw2wannier90 FINISHED
node:  scf FINISHED
node:  seekpath FINISHED
node:  wannier90 FINISHED
node:  wannier90_pp FINISHED
node:  wannier90_pp_parameters FINISHED
is workgraph finished:  True
group outputs:  []
Finalize workgraph Wannier-GaAs



{'group_outputs': {},
 'new_data': {},
 'execution_count': <Int: uuid: 2a440dc8-444d-4a7a-99ba-51216b8781d6 (pk: 15303) value: 0>}

# Compare with parameters with the workchain

In [ ]:
from aiida_wannier90_workflows.workflows import Wannier90BandsWorkChain
from aiida import load_profile, orm
from ase.build import bulk
from copy import deepcopy
from aiida.plugins import DataFactory
from aiida.engine import submit
load_profile()


StructureData = DataFactory('core.structure')
a = 5.68018817933178
structure = StructureData(cell = [[-a/2., 0, a/2.], [0, a/2., a/2.], [-a/2., a/2., 0]])
structure.append_atom(symbols=['Ga'], position=(0., 0., 0.))
structure.append_atom(symbols=['As'], position=(-a/4., a/4., a/4.))
structure.store()


structure = orm.StructureData(ase=bulk("Si"))
pw_code = orm.load_code("qe-7.2-pw@localhost")
projwfc_code = orm.load_code('qe-7.2-projwfc@localhost')
pw2wannier90_code = orm.load_code('qe-7.2-pw2wannier90@localhost')
wannier90_code = orm.load_code('wannier90@localhost')
codes = {
    "pw": pw_code,
    "projwfc": projwfc_code,
    "pw2wannier90": pw2wannier90_code,
    "wannier90": wannier90_code,
}
# Prepare the builder to launch the workchain.
# We use fast protocol to converge faster.
builder = Wannier90BandsWorkChain.get_builder_from_protocol(
    codes,
    structure,
    protocol="fast",
    pseudo_family="SSSP/1.3/PBEsol/efficiency",
)
# builder
# Submit the workchain.
# workchain = submit(builder)
# print(f"Submitted {workchain.process_label}<{workchain.pk}>")

# pk 13512